In [ ]:
# Group project for CSE 6240: Web Search and Text Mining at Georgia Tech
# Author: Kien Tran (github.com/trantrikien239)
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch import Tensor
print(torch.__version__)

2.0.0+cu118


In [ ]:
# Install required packages.
import os
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 102.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 84.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/nightly/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-utud8wcy
  Running command git clone --filter=blob:none --quiet https://github

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/CSE6240/

/content/drive/.shortcut-targets-by-id/1y1Lvxa-gIYQlLQs7Dzoz9XVXO1BPTpP5/CSE6240


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch_geometric import nn
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader

In [ ]:
from models.interaction_func import DotProduct, MLP2LayersV1, MLP2LayersV2
from models.graphsage import GraphSage2Layers, GraphSage3Layers, GraphSageLinkPred,\
    GraphSageLinkPredNoEmb
from models.gat import GAT2Layers, GAT3Layers, GATLinkPred, GATLinkPredNoEmb

In [ ]:
from models.utils import prep_dataset, eval_auc, train, train_early_stop

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
BATCH_SIZE = 2048 * 4

In [ ]:
# Load data
test_edges = pd.read_parquet('./data/graphs/test_edges.parquet')
test_labels = pd.read_parquet('./data/graphs/test_labels_neg.parquet')

val_edges = pd.read_parquet('./data/graphs/val_edges.parquet')
val_labels = pd.read_parquet('./data/graphs/val_labels_neg.parquet')

train_edges = pd.read_parquet('./data/graphs/train_edges.parquet')
train_labels = pd.read_parquet('./data/graphs/train_labels_neg.parquet')

In [ ]:
prod_feat = np.load('./data/feature_emb/products.npy')
user_feat = np.load('./data/feature_agg/train_user_features_norm.npy')

In [ ]:
with open(r"saved_model/node2vec_128_v2.pkl", "rb") as f:
    z = torch.load(f, map_location=torch.device('cpu'))
z.shape

torch.Size([256210, 128])

In [ ]:
enc_user_id = train_edges[["user_id"]].drop_duplicates().sort_values("user_id").reset_index(drop=True).reset_index().rename(columns={"index": "enc_user_id"})
enc_user_id_dict = dict(zip(enc_user_id.user_id, enc_user_id.enc_user_id))
# encode user_id
train_edges["enc_user_id"] = train_edges["user_id"].map(enc_user_id_dict)
val_edges["enc_user_id"] = val_edges["user_id"].map(enc_user_id_dict)
test_edges["enc_user_id"] = test_edges["user_id"].map(enc_user_id_dict)
train_labels["enc_user_id"] = train_labels["user_id"].map(enc_user_id_dict)
val_labels["enc_user_id"] = val_labels["user_id"].map(enc_user_id_dict)
test_labels["enc_user_id"] = test_labels["user_id"].map(enc_user_id_dict)


In [ ]:
train_edges

,user_id,product_id,weight,enc_user_id
0,1,196,0.700000,0
1,1,10258,0.600000,0
2,1,10326,0.100000,0
3,1,12427,0.700000,0
4,1,13032,0.200000,0
...,...,...,...,...
8675716,206209,41665,0.076923,101695
8675717,206209,43961,0.153846,101695
8675718,206209,44325,0.076923,101695
8675719,206209,48697,0.076923,101695


In [ ]:
map_uid = train_edges[["user_id", "enc_user_id"]].drop_duplicates()\
  .sort_values("enc_user_id").reset_index(drop=True)
map_uid["fake_uid"] = map_uid["user_id"] + 50000
map_uid

,user_id,enc_user_id,fake_uid
0,1,0,50001
1,2,1,50002
2,3,2,50003
3,7,3,50007
4,13,4,50013
...,...,...,...
101691,206202,101691,256202
101692,206206,101692,256206
101693,206207,101693,256207
101694,206208,101694,256208


In [ ]:
n2v_user_emb = z[map_uid["fake_uid"],:].detach().cpu().numpy()
n2v_user_emb.shape

(101696, 128)

In [ ]:
n_prod = prod_feat.shape[0]
n2v_prod_emb = z[:n_prod,:].detach().cpu().numpy()
n2v_prod_emb.shape

(49689, 128)

In [ ]:
prod_features = np.hstack([n2v_prod_emb, prod_feat])
user_features = np.hstack([n2v_user_emb, user_feat])

### Prepare dataset and data loaders

In [ ]:
# Data loaders for baseline features
base_train_data = prep_dataset(train_edges, train_labels, user_feat, prod_feat)
base_train_loader = LinkNeighborLoader(
    data=base_train_data,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      base_train_data["user", "buy", "prod"].edge_label_index),
    edge_label=base_train_data["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [ ]:
base_val_data_lite = prep_dataset(val_edges, 
                        val_labels[val_labels["enc_user_id"] < 10_000], 
                        user_feat, prod_feat)
# Define the validation seed edges:
base_val_loader_lite = LinkNeighborLoader(
    data=base_val_data_lite,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      base_val_data_lite["user", "buy", "prod"].edge_label_index),
    edge_label=base_val_data_lite["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE * 4,
    shuffle=False,
)
val_data_large = prep_dataset(val_edges, 
                        val_labels[val_labels["enc_user_id"] >= 10_000], 
                        user_feat, prod_feat)
# Define the validation seed edges:
base_val_loader_large = LinkNeighborLoader(
    data=val_data_large,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      val_data_large["user", "buy", "prod"].edge_label_index),
    edge_label=val_data_large["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE * 4,
    shuffle=False,
)


In [ ]:
n_user, base_user_feat_size = user_feat.shape
n_prod, base_prod_feat_size = prod_feat.shape

In [ ]:
# Data loaders for advanced features
train_data = prep_dataset(train_edges, train_labels, user_features, prod_features)
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      train_data["user", "buy", "prod"].edge_label_index),
    edge_label=train_data["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

print(sampled_data["user", "buy", "prod"].edge_label_index.size(1))
print(sampled_data["user", "buy", "prod"].edge_label.min())
print(sampled_data["user", "buy", "prod"].edge_label.max())

Sampled mini-batch:
HeteroData(
  user={
    node_id=[85936],
    x=[85936, 150],
    n_id=[85936]
  },
  prod={
    node_id=[40418],
    x=[40418, 896],
    n_id=[40418]
  },
  (user, buy, prod)={
    edge_index=[2, 468335],
    edge_label_index=[2, 8192],
    edge_label=[8192],
    e_id=[468335],
    input_id=[8192]
  },
  (prod, rev_buy, user)={
    edge_index=[2, 1067969],
    e_id=[1067969]
  }
)
8192
tensor(0.)
tensor(1.)


In [ ]:
val_data_lite = prep_dataset(val_edges, 
                        val_labels[val_labels["enc_user_id"] < 10_000], 
                        user_features, prod_features)
# Define the validation seed edges:
val_loader_lite = LinkNeighborLoader(
    data=val_data_lite,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      val_data_lite["user", "buy", "prod"].edge_label_index),
    edge_label=val_data_lite["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE * 4,
    shuffle=False,
)
val_data_large = prep_dataset(val_edges, 
                        val_labels[val_labels["enc_user_id"] >= 10_000], 
                        user_features, prod_features)
# Define the validation seed edges:
val_loader_large = LinkNeighborLoader(
    data=val_data_large,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      val_data_large["user", "buy", "prod"].edge_label_index),
    edge_label=val_data_large["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE * 4,
    shuffle=False,
)


In [ ]:
n_user, user_feat_size = user_features.shape
n_prod, prod_feat_size = prod_features.shape
train_meta_data = train_data.metadata()
train_meta_data

(['user', 'prod'], [('user', 'buy', 'prod'), ('prod', 'rev_buy', 'user')])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Exp 1: Feature set x NCF

Experiment with GraphSAGE, try combinations of:
- Baseline Features vs Advanced Features (Baseline + Node2Vec embeddings)
- Vallina GraphSAGE vs (GraphSAGE + NCF-inspired embeddings)

In [ ]:
gs_base_noemb = GraphSageLinkPredNoEmb(hidden_channels=256, 
                 user_feat_size=base_user_feat_size, 
                 prod_feat_size=base_prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=3,
                 interaction_func="mlp2layers_v1")

gs_base_emb = GraphSageLinkPred(hidden_channels=256, 
                 n_user=n_user, n_prod=n_prod, 
                 user_feat_size=base_user_feat_size, 
                 prod_feat_size=base_prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=3,
                 interaction_func="mlp2layers_v1")

gs_adv_noemb = GraphSageLinkPredNoEmb(hidden_channels=256, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=3,
                 interaction_func="mlp2layers_v1")

gs_adv_emb = GraphSageLinkPred(hidden_channels=256, 
                 n_user=n_user, n_prod=n_prod, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=3,
                 interaction_func="mlp2layers_v1")

In [ ]:
gs_base_noemb, df_stats = train_early_stop(
    gs_base_noemb, base_train_loader, base_val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_base_noemb, base_val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))

  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.5242, AUC: 0.8757, Precision: 0.7715, Recall: 0.8250, F1: 0.7973


  0%|          | 0/81 [00:00<?, ?it/s]

...

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 015, Loss: 0.3384, AUC: 0.9201, Precision: 0.8097, Recall: 0.8828, F1: 0.8447
Early stopping at epoch 15


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.921811370009544
              precision    recall  f1-score   support

         0.0       0.87      0.79      0.83    298582
         1.0       0.81      0.88      0.85    298582

    accuracy                           0.84    597164
   macro avg       0.84      0.84      0.84    597164
weighted avg       0.84      0.84      0.84    597164



In [ ]:
df_stats.to_parquet("stats/gs_base_noemb.parquet")
df_stats.tail()

,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
10,10,0.357136,0.913243,0.796652,0.886722,0.839278
11,11,0.353589,0.914026,0.810435,0.864805,0.836738
12,12,0.349590,0.917622,0.810198,0.872865,0.840365
13,13,0.346458,0.919140,0.817375,0.864836,0.840436
14,14,0.341329,0.920490,0.820074,0.864743,0.841816


In [ ]:
gs_base_emb, df_stats = train_early_stop(
    gs_base_emb, base_train_loader, base_val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_base_emb, base_val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gs_base_emb.parquet")
df_stats.tail()

  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.5006, AUC: 0.9030, Precision: 0.7987, Recall: 0.8399, F1: 0.8188


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 006, Loss: 0.2766, AUC: 0.9307, Precision: 0.8251, Recall: 0.8914, F1: 0.8570
Early stopping at epoch 6


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9308854058463027
              precision    recall  f1-score   support

         0.0       0.88      0.81      0.84    298582
         1.0       0.82      0.89      0.86    298582

    accuracy                           0.85    597164
   macro avg       0.85      0.85      0.85    597164
weighted avg       0.85      0.85      0.85    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
1,1,0.352375,0.921884,0.878484,0.756177,0.812755
2,2,0.320370,0.928752,0.831350,0.870757,0.850597
3,3,0.303344,0.930788,0.823964,0.889202,0.855341
4,4,0.294643,0.931244,0.828172,0.889636,0.857805
5,5,0.284756,0.931668,0.828936,0.888272,0.857579


In [ ]:
print("gs_base_noemb num parameters:", sum(p.numel() for p in gs_base_noemb.parameters()))
print("gs_base_emb num parameters:", sum(p.numel() for p in gs_base_emb.parameters()))

gs_base_noemb num parameters: 1056769
gs_base_emb num parameters: 39811329


In [ ]:
gs_adv_noemb, df_stats = train_early_stop(
    gs_adv_noemb, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_adv_noemb, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gs_adv_noemb.parquet")
df_stats.tail()

  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4321, AUC: 0.9242, Precision: 0.8416, Recall: 0.8559, F1: 0.8487


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 014, Loss: 0.2659, AUC: 0.9505, Precision: 0.8632, Recall: 0.9062, F1: 0.8842
Early stopping at epoch 14


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9501611890190083
              precision    recall  f1-score   support

         0.0       0.90      0.85      0.88    298582
         1.0       0.86      0.91      0.88    298582

    accuracy                           0.88    597164
   macro avg       0.88      0.88      0.88    597164
weighted avg       0.88      0.88      0.88    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
9,9,0.278626,0.949139,0.864567,0.899464,0.881670
10,10,0.276086,0.949184,0.862839,0.901572,0.881780
11,11,0.273366,0.949725,0.864653,0.902502,0.883172
12,12,0.270659,0.950234,0.859471,0.910655,0.884323
13,13,0.268385,0.950985,0.863630,0.908609,0.885549


In [ ]:
gs_adv_emb, df_stats = train_early_stop(
    gs_adv_emb, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_adv_emb, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gs_adv_emb.parquet")
df_stats.tail()

  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4984, AUC: 0.9039, Precision: 0.8215, Recall: 0.7993, F1: 0.8103


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 005, Loss: 0.2753, AUC: 0.9359, Precision: 0.8393, Recall: 0.8916, F1: 0.8646
Early stopping at epoch 5


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.936256474882958
              precision    recall  f1-score   support

         0.0       0.89      0.83      0.85    298582
         1.0       0.84      0.89      0.86    298582

    accuracy                           0.86    597164
   macro avg       0.86      0.86      0.86    597164
weighted avg       0.86      0.86      0.86    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
0,0,0.498448,0.903909,0.821487,0.799330,0.810257
1,1,0.344648,0.930033,0.853085,0.847320,0.850193
2,2,0.311799,0.934506,0.848776,0.870695,0.859596
3,3,0.297435,0.936121,0.853718,0.868804,0.861195
4,4,0.285409,0.936593,0.843430,0.883932,0.863206


# Exp 2: GraphSAGE: Depth and Size

Since we have determined that gs_adv_noemb is the best setting so far, let's experiments with a few variations of this model. Namely: Depth and Hidden size

In [ ]:
# Depth try 2 gnn layers instead of the default 3
gs_adv_noemb_2gnn = GraphSageLinkPredNoEmb(hidden_channels=256, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2,
                 interaction_func="mlp2layers_v1")
gs_adv_noemb_2gnn, df_stats = train_early_stop(
    gs_adv_noemb_2gnn, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_adv_noemb_2gnn, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gs_adv_noemb_2gnn.parquet")
df_stats.tail()

  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4198, AUC: 0.9289, Precision: 0.8353, Recall: 0.8740, F1: 0.8542


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 014, Loss: 0.2602, AUC: 0.9514, Precision: 0.8608, Recall: 0.9141, F1: 0.8867
Early stopping at epoch 14


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9507084269152051
              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88    298582
         1.0       0.86      0.91      0.88    298582

    accuracy                           0.88    597164
   macro avg       0.88      0.88      0.88    597164
weighted avg       0.88      0.88      0.88    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
9,9,0.273253,0.950337,0.863869,0.905540,0.884214
10,10,0.271210,0.950357,0.858556,0.912453,0.884684
11,11,0.268074,0.950871,0.866512,0.902750,0.884260
12,12,0.265689,0.951491,0.859535,0.914933,0.886369
13,13,0.263183,0.951765,0.862571,0.911988,0.886592


In [ ]:
print("gs_adv_noemb num parameters:", sum(p.numel() for p in gs_adv_noemb.parameters()))
print("gs_adv_noemb_2gnn num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn.parameters()))

gs_adv_noemb num parameters: 1122305
gs_adv_noemb_2gnn num parameters: 859649


The two-layer version GraphSAGE performs slightly better than the 3-layer version. This result suggests that the deeper version (3 layers) encorage over-fitting to the training set which harm generalization.

Considering the occam's razor principle, we will proceed with 2 layers models.

In [ ]:
# Hidden size: Try 128 and 512 for hidden size instead of default 256
gs_adv_noemb_2gnn_128d = GraphSageLinkPredNoEmb(hidden_channels=128, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2,
                 interaction_func="mlp2layers_v1")
gs_adv_noemb_2gnn_128d, df_stats = train_early_stop(
    gs_adv_noemb_2gnn_128d, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_adv_noemb_2gnn_128d, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gs_adv_noemb_2gnn_128d.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4296, AUC: 0.9295, Precision: 0.8346, Recall: 0.8781, F1: 0.8558


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 015, Loss: 0.2669, AUC: 0.9508, Precision: 0.8728, Recall: 0.8939, F1: 0.8832
Early stopping at epoch 15


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9504605318465849
              precision    recall  f1-score   support

         0.0       0.89      0.87      0.88    298582
         1.0       0.87      0.89      0.88    298582

    accuracy                           0.88    597164
   macro avg       0.88      0.88      0.88    597164
weighted avg       0.88      0.88      0.88    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
10,10,0.275609,0.950138,0.858329,0.910190,0.883499
11,11,0.273525,0.950355,0.870182,0.894380,0.882115
12,12,0.271577,0.950561,0.866937,0.902037,0.884139
13,13,0.269527,0.950917,0.864813,0.901355,0.882706
14,14,0.268606,0.951506,0.868624,0.900642,0.884343


In [ ]:
gs_adv_noemb_2gnn_512d = GraphSageLinkPredNoEmb(hidden_channels=512, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2,
                 interaction_func="mlp2layers_v1")
gs_adv_noemb_2gnn_512d, df_stats = train_early_stop(
    gs_adv_noemb_2gnn_512d, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gs_adv_noemb_2gnn_512d, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gs_adv_noemb_2gnn_512d.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4087, AUC: 0.9386, Precision: 0.8593, Recall: 0.8700, F1: 0.8646


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 009, Loss: 0.2676, AUC: 0.9510, Precision: 0.8633, Recall: 0.9058, F1: 0.8840
Early stopping at epoch 9


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.950500113883344
              precision    recall  f1-score   support

         0.0       0.90      0.86      0.88    298582
         1.0       0.86      0.91      0.88    298582

    accuracy                           0.88    597164
   macro avg       0.88      0.88      0.88    597164
weighted avg       0.88      0.88      0.88    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
4,4,0.288583,0.947036,0.859050,0.897666,0.877933
5,5,0.282764,0.948543,0.868485,0.890535,0.879372
6,6,0.278197,0.949553,0.863631,0.902533,0.882653
7,7,0.274247,0.950465,0.877117,0.886443,0.881755
8,8,0.270922,0.951203,0.868677,0.900239,0.884176


In [ ]:
print("gs_adv_noemb_2gnn_128d num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn_128d.parameters()))
print("gs_adv_noemb_2gnn num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn.parameters()))
print("gs_adv_noemb_2gnn_512d num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn_512d.parameters()))

gs_adv_noemb_2gnn_128d num parameters: 299009
gs_adv_noemb_2gnn num parameters: 859649
gs_adv_noemb_2gnn_512d num parameters: 2767361


A hidden dimension of 256 produce the best AUC on the validation set. Although 128d and 512d version doesn't perform much worse either.

In [ ]:
with open(r"saved_model/gs_adv_noemb.pkl", "wb") as f:
    torch.save(gs_adv_noemb, f=f)
with open(r"saved_model/gs_adv_noemb_2gnn.pkl", "wb") as f:
    torch.save(gs_adv_noemb_2gnn, f=f)
with open(r"saved_model/gs_adv_noemb_2gnn_128d.pkl", "wb") as f:
    torch.save(gs_adv_noemb_2gnn_128d, f=f)
with open(r"saved_model/gs_adv_noemb_2gnn_512d.pkl", "wb") as f:
    torch.save(gs_adv_noemb_2gnn_512d, f=f)


# Exp 3: Graph Attention Network

With insights from GraphSAGE, we made a judgement call to focus on Advanced features, pure Graph Attention Network (without NCF-style internal embedding learning), and only message-passing GNN layers. That being said, there are a few hyperparameters we can experiment with to understand the effect of the network's architecture on it's learning effectiveness. We experiment with 2 aspect: Hidden size and the number of heads.

In [ ]:
gat_128d_1h = GATLinkPredNoEmb(hidden_channels=128, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2, num_heads=1,
                 interaction_func="mlp2layers_v1")
gat_128d_1h, df_stats = train_early_stop(
    gat_128d_1h, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gat_128d_1h, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gat_128d_1h.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.5176, AUC: 0.9032, Precision: 0.8390, Recall: 0.7770, F1: 0.8068


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 013, Loss: 0.3052, AUC: 0.9401, Precision: 0.8484, Recall: 0.8953, F1: 0.8712
Early stopping at epoch 13


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9396213379180625
              precision    recall  f1-score   support

         0.0       0.89      0.84      0.86    298582
         1.0       0.85      0.90      0.87    298582

    accuracy                           0.87    597164
   macro avg       0.87      0.87      0.87    597164
weighted avg       0.87      0.87      0.87    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
8,8,0.313612,0.938004,0.847233,0.889388,0.867799
9,9,0.311015,0.938414,0.853469,0.879530,0.866303
10,10,0.309300,0.939374,0.862001,0.869083,0.865527
11,11,0.308490,0.939883,0.849932,0.891062,0.870011
12,12,0.306987,0.940142,0.842895,0.900487,0.870740


In [ ]:
gat_128d_2h = GATLinkPredNoEmb(hidden_channels=128, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2, num_heads=2,
                 interaction_func="mlp2layers_v1")
gat_128d_2h, df_stats = train_early_stop(
    gat_128d_2h, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gat_128d_2h, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gat_128d_2h.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.5176, AUC: 0.9042, Precision: 0.8678, Recall: 0.7384, F1: 0.7979


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 005, Loss: 0.3090, AUC: 0.9394, Precision: 0.8402, Recall: 0.9005, F1: 0.8693
Early stopping at epoch 5


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9397369431063298
              precision    recall  f1-score   support

         0.0       0.89      0.83      0.86    298582
         1.0       0.84      0.90      0.87    298582

    accuracy                           0.86    597164
   macro avg       0.87      0.86      0.86    597164
weighted avg       0.87      0.86      0.86    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
0,0,0.517607,0.904249,0.867850,0.738413,0.797916
1,1,0.351936,0.930595,0.842873,0.872896,0.857621
2,2,0.326312,0.935685,0.843196,0.887032,0.864559
3,3,0.316100,0.938010,0.843567,0.890535,0.866415
4,4,0.310804,0.939514,0.856398,0.876337,0.866253


In [ ]:
gat_256d_2h = GATLinkPredNoEmb(hidden_channels=256, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2, num_heads=2,
                 interaction_func="mlp2layers_v1")
gat_256d_2h, df_stats = train_early_stop(
    gat_256d_2h, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gat_256d_2h, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gat_256d_2h.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4707, AUC: 0.9268, Precision: 0.8489, Recall: 0.8570, F1: 0.8529


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 003, Loss: 0.3120, AUC: 0.9368, Precision: 0.8799, Recall: 0.8218, F1: 0.8499
Early stopping at epoch 3


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9371042581254536
              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86    298582
         1.0       0.88      0.82      0.85    298582

    accuracy                           0.86    597164
   macro avg       0.86      0.86      0.86    597164
weighted avg       0.86      0.86      0.86    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
0,0,0.470697,0.926768,0.848865,0.857023,0.852925
1,1,0.331110,0.934178,0.860365,0.857271,0.858815
2,2,0.320132,0.937018,0.855900,0.872059,0.863904


In [ ]:
gat_256d_4h = GATLinkPredNoEmb(hidden_channels=256, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2, num_heads=4,
                 interaction_func="mlp2layers_v1")
gat_256d_4h, df_stats = train_early_stop(
    gat_256d_4h, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gat_256d_4h, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gat_256d_4h.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4780, AUC: 0.9271, Precision: 0.8499, Recall: 0.8496, F1: 0.8497


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 014, Loss: 0.2859, AUC: 0.9465, Precision: 0.8481, Recall: 0.9138, F1: 0.8798
Early stopping at epoch 14


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9465907407837573
              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87    298582
         1.0       0.85      0.91      0.88    298582

    accuracy                           0.87    597164
   macro avg       0.88      0.87      0.87    597164
weighted avg       0.88      0.87      0.87    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
9,9,0.291847,0.945730,0.866429,0.881390,0.873845
10,10,0.290962,0.945776,0.856881,0.896860,0.876414
11,11,0.289783,0.946152,0.870104,0.876740,0.873410
12,12,0.288462,0.946414,0.854062,0.904951,0.878771
13,13,0.287353,0.946520,0.853858,0.904920,0.878648


In [ ]:
gat_512d_4h = GATLinkPredNoEmb(hidden_channels=512, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2, num_heads=4,
                 interaction_func="mlp2layers_v1")
gat_512d_4h, df_stats = train_early_stop(
    gat_512d_4h, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gat_512d_4h, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gat_512d_4h.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4679, AUC: 0.9261, Precision: 0.8422, Recall: 0.8580, F1: 0.8500


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 006, Loss: 0.2955, AUC: 0.9443, Precision: 0.8491, Recall: 0.9011, F1: 0.8744
Early stopping at epoch 6


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.944198776796188
              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87    298582
         1.0       0.85      0.90      0.87    298582

    accuracy                           0.87    597164
   macro avg       0.87      0.87      0.87    597164
weighted avg       0.87      0.87      0.87    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
1,1,0.325265,0.937850,0.849975,0.882754,0.866054
2,2,0.309459,0.940974,0.846089,0.895558,0.870120
3,3,0.304310,0.942354,0.855009,0.889016,0.871681
4,4,0.300442,0.943306,0.861821,0.882847,0.872207
5,5,0.297716,0.944451,0.864122,0.880677,0.872321


In [33]:
gat_512d_8h = GATLinkPredNoEmb(hidden_channels=512, 
                 user_feat_size=user_feat_size, 
                 prod_feat_size=prod_feat_size, 
                 dataset_metadata=train_meta_data,
                 n_gnn_layers=2, num_heads=8,
                 interaction_func="mlp2layers_v1")
gat_512d_8h, df_stats = train_early_stop(
    gat_512d_8h, train_loader, val_loader_lite, device, 
    epochs=20, return_stats=True)
auc1, gt1, pred1 = eval_auc(gat_512d_8h, val_loader_large, device)
print("AUC: ", auc1)
pred_proba = torch.sigmoid(torch.tensor(pred1)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt1, pred_binary))
df_stats.to_parquet("stats/gat_512d_8h.parquet")
df_stats.tail()


  0%|          | 0/81 [00:00<?, ?it/s]

Evaluating on the lite validation set...


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 000, Loss: 0.4696, AUC: 0.9317, Precision: 0.8422, Recall: 0.8778, F1: 0.8596


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 005, Loss: 0.2948, AUC: 0.9448, Precision: 0.8515, Recall: 0.9013, F1: 0.8756
Early stopping at epoch 5


  0%|          | 0/19 [00:00<?, ?it/s]

AUC:  0.9446599423615921
              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87    298582
         1.0       0.85      0.90      0.88    298582

    accuracy                           0.87    597164
   macro avg       0.87      0.87      0.87    597164
weighted avg       0.87      0.87      0.87    597164



,epoch,train_loss,val_auc,val_precision,val_recall,val_f1
0,0,0.469568,0.931729,0.842177,0.877763,0.859602
1,1,0.318624,0.938609,0.850742,0.881731,0.865959
2,2,0.306215,0.941736,0.853957,0.885699,0.869539
3,3,0.301101,0.943544,0.846852,0.903060,0.874053
4,4,0.297133,0.944887,0.849243,0.901293,0.874494


# Test performance

In [34]:
test_data = prep_dataset(test_edges, 
                        test_labels, 
                        user_features, prod_features)
# Define the validation seed edges:
test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), 
                      test_data["user", "buy", "prod"].edge_label_index),
    edge_label=test_data["user", "buy", "prod"].edge_label,
    batch_size=BATCH_SIZE * 4,
    shuffle=False,
)


In [ ]:
# GraphSAGE models
# gs_adv_noemb
# gs_adv_noemb_2gnn
# gs_adv_noemb_2gnn_128d
# gs_adv_noemb_2gnn_512d


In [ ]:
print("gs_adv_noemb num parameters:", sum(p.numel() for p in gs_adv_noemb.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gs_adv_noemb, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gs_adv_noemb num parameters: 1122305
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9500063038576192
              precision    recall  f1-score   support

         0.0       0.90      0.86      0.88    328203
         1.0       0.86      0.90      0.88    328203

    accuracy                           0.88    656406
   macro avg       0.88      0.88      0.88    656406
weighted avg       0.88      0.88      0.88    656406



In [ ]:
print("gs_adv_noemb_2gnn num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gs_adv_noemb_2gnn, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gs_adv_noemb_2gnn num parameters: 859649
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9504444586320195
              precision    recall  f1-score   support

         0.0       0.90      0.85      0.88    328203
         1.0       0.86      0.91      0.88    328203

    accuracy                           0.88    656406
   macro avg       0.88      0.88      0.88    656406
weighted avg       0.88      0.88      0.88    656406



In [ ]:
print("gs_adv_noemb_2gnn_128d num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn_128d.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gs_adv_noemb_2gnn_128d, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gs_adv_noemb_2gnn_128d num parameters: 299009
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9501906084608093
              precision    recall  f1-score   support

         0.0       0.89      0.87      0.88    328203
         1.0       0.87      0.89      0.88    328203

    accuracy                           0.88    656406
   macro avg       0.88      0.88      0.88    656406
weighted avg       0.88      0.88      0.88    656406



In [ ]:
print("gs_adv_noemb_2gnn_512d num parameters:", sum(p.numel() for p in gs_adv_noemb_2gnn_512d.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gs_adv_noemb_2gnn_512d, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gs_adv_noemb_2gnn_512d num parameters: 2767361
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9502408344139297
              precision    recall  f1-score   support

         0.0       0.90      0.86      0.88    328203
         1.0       0.86      0.90      0.88    328203

    accuracy                           0.88    656406
   macro avg       0.88      0.88      0.88    656406
weighted avg       0.88      0.88      0.88    656406



In [ ]:
# Graph Attention Models

In [36]:
print("gat_128d_1h num parameters:", sum(p.numel() for p in gat_128d_1h.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gat_128d_1h, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gat_128d_1h num parameters: 300545
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9395048443990365
              precision    recall  f1-score   support

         0.0       0.89      0.84      0.86    328203
         1.0       0.85      0.89      0.87    328203

    accuracy                           0.87    656406
   macro avg       0.87      0.87      0.87    656406
weighted avg       0.87      0.87      0.87    656406



In [37]:
print("gat_256d_4h num parameters:", sum(p.numel() for p in gat_256d_4h.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gat_256d_4h, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gat_256d_4h num parameters: 862721
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9461773800112936
              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87    328203
         1.0       0.85      0.91      0.88    328203

    accuracy                           0.87    656406
   macro avg       0.88      0.87      0.87    656406
weighted avg       0.88      0.87      0.87    656406



In [35]:
print("gat_512d_8h num parameters:", sum(p.numel() for p in gat_512d_8h.parameters()))
print("Performance on test set:")
auc2, gt2, pred2 = eval_auc(gat_512d_8h, test_loader, device)
print("AUC: ", auc2)
pred_proba = torch.sigmoid(torch.tensor(pred2)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(gt2, pred_binary))

gat_512d_8h num parameters: 2773505
Performance on test set:


  0%|          | 0/21 [00:00<?, ?it/s]

AUC:  0.9441353539310111
              precision    recall  f1-score   support

         0.0       0.89      0.84      0.87    328203
         1.0       0.85      0.90      0.87    328203

    accuracy                           0.87    656406
   macro avg       0.87      0.87      0.87    656406
weighted avg       0.87      0.87      0.87    656406



In [38]:
with open(r"saved_model/gat_128d_1h.pkl", "wb") as f:
    torch.save(gat_128d_1h, f=f)
with open(r"saved_model/gat_256d_4h.pkl", "wb") as f:
    torch.save(gat_256d_4h, f=f)
with open(r"saved_model/gat_512d_8h.pkl", "wb") as f:
    torch.save(gat_512d_8h, f=f)
    

# References
1. [Pytorch Geometric's tutorial for Heterogeneous graphs](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html)
2. [Inductive Representation Learning on Large Graphs by Hamilton et al - GraphSAGE paper](https://arxiv.org/abs/1706.02216)